## Import architectures and necessary libraries

In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
from vgg16 import VGG16
from vgg19 import VGG19
from glob import glob
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

2023-05-04 18:12:48.900067: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-04 18:12:49.473659: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Preprocessing images for train model(Imagenet2012 validation dataset)

In [ ]:
labels = []
images = []

imagesfiles = [i for i in glob('data/*.JPEG')]
for i in imagesfiles:
    image = cv2.imread(i)/255.0
    image = cv2.resize(image, (224,224))
    images.append(image)
images = np.array(images)

with open('data/imagenet_2012_validation_synset_labels.txt') as f:
    labelstxt = f.read().split('\n')[:-1]
with open('data/labels.txt') as f:
    labs = f.read()
    
for i in labelstxt:
    bosh = labs.find(i)
    labels.append(int(labs[bosh+10:bosh+10+labs[bosh+10:].find(' ')])-1)
    
images = np.array(images)
labels = np.array(labels)

In [3]:
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=100)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(40000, 224, 224, 3) (40000,)
(10000, 224, 224, 3) (10000,)


In [10]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y

In [5]:
train_gen = DataGenerator(x_train, y_train, 32)
test_gen = DataGenerator(x_test, y_test, 32)

In [6]:
class Model(keras.models.Model):
    def __init__(self):
        super(Model, self).__init__()
        self.base_model = VGG19()
    def call(self, x):
        x = self.base_model(x)
        return x
    
model = Model()

2023-05-04 16:35:37.053531: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-04 16:35:37.053748: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-04 16:35:37.078500: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the 

In [7]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [8]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        # training=True is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

@tf.function
def test_step(images, labels):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)
    
    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [ ]:
epochs = 10

for epoch in range(epochs):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    for train_images, train_labels in train_gen:
        train_step(train_images, train_labels)

    for test_images, test_labels in test_gen:
        test_step(test_images, test_labels)
    if epoch % 1 == 0:
        print(
          f'Epoch {epoch}, '
          f'Loss: {train_loss.result()}, '
          f'Accuracy: {train_accuracy.result() * 100}%, '
          f'Test Loss: {test_loss.result()}, '
          f'Test Accuracy: {test_accuracy.result() * 100}%'
      )

/home/airi/anaconda3/lib/python3.9/site-packages/keras/backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


Epoch 0, Loss: 6.907754898071289, Accuracy: 0.07000000029802322%, Test Loss: 6.907752513885498, Test Accuracy: 0.10999999940395355%


## Finetune architectures

In [34]:
train = tfds.load('oxford_iiit_pet', split='train')
test = tfds.load('oxford_iiit_pet', split='test')
# valid = tfds.load('lfw', split='validation')

x_train, y_train, y_train2, x_test, y_test, y_test2 = [], [], [], [], [], []
for i in train:
    a = i['image'].numpy()
    x_train.append(cv2.resize(a, (224, 224))/255.0)
    y_train.append(i['label'].numpy())
    y_train2.append(i['species'].numpy())
    
for i in test:
    a = i['image'].numpy()
    x_test.append(cv2.resize(a, (224, 224))/255.0)
    y_test.append(i['label'].numpy())
    y_test2.append(i['species'].numpy())
    
x_train = np.array(x_train)
y_train = np.array(y_train)
y_train2 = np.array(y_train2)
y_train2 = np.expand_dims(y_train2, axis=1)

x_test = np.array(x_test)
y_test = np.array(y_test)
y_test2 = np.array(y_test2)
y_test2 = np.expand_dims(y_test2, axis=1)

x_train.shape, y_train.shape, y_train2.shape, x_test.shape, y_test.shape, y_test2.shape

2023-05-04 18:43:58.228796: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int64 and shape [4]
	 [[{{node Placeholder/_4}}]]
2023-05-04 18:43:58.229079: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int64 and shape [4]
	 [[{{node Placeholder/_4}}]]
Corrupt JPEG data: 240 extraneous bytes before marker 0xd9
Corrupt JPEG data: premature end of data segment
2023-05-04 18:43:59.893363: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUM

((3680, 224, 224, 3),
 (3680,),
 (3680, 1),
 (3669, 224, 224, 3),
 (3669,),
 (3669, 1))

In [35]:
# classification for two classes
class Model1(keras.models.Model):
    def __init__(self):
        super(Model1, self).__init__()
        self.base_model = VGG19(include_top=False)
        self.flatten = keras.layers.Flatten()
        self.fc1 = keras.layers.Dense(1000, activation='relu')
        self.fc2 = keras.layers.Dense(256, activation='relu')
        self.out = keras.layers.Dense(1, activation='sigmoid')
        
    def call(self, x):
        x = self.base_model(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.out(x)
        return x
    
# classification for 37 classes
class Model2(keras.models.Model):
    def __init__(self):
        super(Model2, self).__init__()
        self.base_model = VGG19(include_top=False)
        self.flatten = keras.layers.Flatten()
        self.fc1 = keras.layers.Dense(1000, activation='relu')
        self.fc2 = keras.layers.Dense(256, activation='relu')
        self.out = keras.layers.Dense(37, activation='softmax')
        
    def call(self, x):
        x = self.base_model(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.out(x)
        return x
    
model1 = Model1()
model2 = Model2()

In [36]:
# loss and optimizer
loss_object1 = tf.keras.losses.BinaryCrossentropy(from_logits=True)
optimizer1 = tf.keras.optimizers.Adam(learning_rate=0.001)

train_loss1 = tf.keras.metrics.Mean(name='train_loss1')
train_accuracy1 = tf.keras.metrics.BinaryAccuracy(name='train_accuracy1')

test_loss1 = tf.keras.metrics.Mean(name='test_loss1')
test_accuracy1 = tf.keras.metrics.BinaryAccuracy(name='test_accuracy1')

# optimize functions
@tf.function
def train_step1(images, labels):
    with tf.GradientTape() as tape:
        predictions = model1(images, training=True)
        loss = loss_object1(labels, predictions)
    gradients = tape.gradient(loss, model1.trainable_variables)
    optimizer1.apply_gradients(zip(gradients, model1.trainable_variables))

    train_loss1(loss)
    train_accuracy1(labels, predictions)

@tf.function
def test_step1(images, labels):
    predictions = model1(images, training=False)
    t_loss = loss_object1(labels, predictions)
    
    test_loss1(t_loss)
    test_accuracy1(labels, predictions)

In [41]:
# loss and optimizer
loss_object2 = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer2 = tf.keras.optimizers.Adam(learning_rate=0.001)

train_loss2 = tf.keras.metrics.Mean(name='train_loss2')
train_accuracy2 = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy2')

test_loss2 = tf.keras.metrics.Mean(name='test_loss2')
test_accuracy2 = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy2')

# optimize functions
@tf.function
def train_step2(images, labels):
    with tf.GradientTape() as tape:
        predictions = model2(images, training=True)
        loss = loss_object2(labels, predictions)
    gradients = tape.gradient(loss, model2.trainable_variables)
    optimizer2.apply_gradients(zip(gradients, model2.trainable_variables))

    train_loss2(loss)
    train_accuracy2(labels, predictions)

@tf.function
def test_step2(images, labels):
    predictions = model2(images, training=False)
    t_loss = loss_object2(labels, predictions)
    
    test_loss2(t_loss)
    test_accuracy2(labels, predictions)

In [42]:
# for 2 classes
train_gen1 = DataGenerator(x_train, y_train2, 32)
test_gen1 = DataGenerator(x_test, y_test2, 32)

# for 37 classes
train_gen2 = DataGenerator(x_train, y_train, 32)
test_gen2 = DataGenerator(x_test, y_test, 32)

### Train 2 classes model

In [39]:
epochs = 10

for epoch in range(epochs):
    # Reset the metrics at the start of the next epoch
    train_loss1.reset_states()
    train_accuracy1.reset_states()
    test_loss1.reset_states()
    test_accuracy1.reset_states()

    for train_images, train_labels in train_gen1:
        train_step1(train_images, train_labels)

    for test_images, test_labels in test_gen1:
        test_step1(test_images, test_labels)
    if epoch % 1 == 0:
        print(
          f'Epoch {epoch}, '
          f'Loss: {train_loss1.result()}, '
          f'Accuracy: {train_accuracy1.result() * 100}%, '
          f'Test Loss: {test_loss1.result()}, '
          f'Test Accuracy: {test_accuracy1.result() * 100}%'
      )

Epoch 0, Loss: 0.630222499370575, Accuracy: 67.4456558227539%, Test Loss: 0.6200920343399048, Test Accuracy: 67.75688171386719%
Epoch 1, Loss: 0.6221380233764648, Accuracy: 67.71739196777344%, Test Loss: 0.6139532327651978, Test Accuracy: 67.75688171386719%
Epoch 2, Loss: 0.6167575716972351, Accuracy: 67.71739196777344%, Test Loss: 0.6081777811050415, Test Accuracy: 67.75688171386719%
Epoch 3, Loss: 0.6101758480072021, Accuracy: 67.71739196777344%, Test Loss: 0.5999705195426941, Test Accuracy: 67.75688171386719%
Epoch 4, Loss: 0.5998147130012512, Accuracy: 67.63587188720703%, Test Loss: 0.5859784483909607, Test Accuracy: 67.75688171386719%
Epoch 5, Loss: 0.5841957926750183, Accuracy: 68.23369598388672%, Test Loss: 0.5737913846969604, Test Accuracy: 68.43827056884766%
Epoch 6, Loss: 0.569035530090332, Accuracy: 69.4565200805664%, Test Loss: 0.5539213418960571, Test Accuracy: 71.13655090332031%
Epoch 7, Loss: 0.5535606145858765, Accuracy: 71.08695220947266%, Test Loss: 0.5452743172645569

### Train 37 classes model

In [43]:
epochs = 10

for epoch in range(epochs):
    # Reset the metrics at the start of the next epoch
    train_loss2.reset_states()
    train_accuracy2.reset_states()
    test_loss2.reset_states()
    test_accuracy2.reset_states()

    for train_images, train_labels in train_gen2:
        train_step2(train_images, train_labels)

    for test_images, test_labels in test_gen2:
        test_step2(test_images, test_labels)
    if epoch % 1 == 0:
        print(
          f'Epoch {epoch}, '
          f'Loss: {train_loss2.result()}, '
          f'Accuracy: {train_accuracy2.result() * 100}%, '
          f'Test Loss: {test_loss2.result()}, '
          f'Test Accuracy: {test_accuracy2.result() * 100}%'
      )

/home/airi/anaconda3/lib/python3.9/site-packages/keras/backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


Epoch 0, Loss: 3.612581729888916, Accuracy: 2.2282607555389404%, Test Loss: 3.610687017440796, Test Accuracy: 2.7255382537841797%
Epoch 1, Loss: 3.609954595565796, Accuracy: 3.04347825050354%, Test Loss: 3.6032521724700928, Test Accuracy: 3.2706458568573%
Epoch 2, Loss: 3.5751006603240967, Accuracy: 4.1032609939575195%, Test Loss: 3.530716896057129, Test Accuracy: 4.715181350708008%
Epoch 3, Loss: 3.492347478866577, Accuracy: 4.67391300201416%, Test Loss: 3.480903148651123, Test Accuracy: 5.0695013999938965%
Epoch 4, Loss: 3.4601118564605713, Accuracy: 4.918478488922119%, Test Loss: 3.4681153297424316, Test Accuracy: 5.396565914154053%
Epoch 5, Loss: 3.4473304748535156, Accuracy: 5.1902174949646%, Test Loss: 3.463289499282837, Test Accuracy: 5.696374893188477%
Epoch 6, Loss: 3.4391191005706787, Accuracy: 5.2717390060424805%, Test Loss: 3.46030855178833, Test Accuracy: 5.532842636108398%
Epoch 7, Loss: 3.43288516998291, Accuracy: 5.5163044929504395%, Test Loss: 3.457617998123169, Test A